In [1]:
from sklearn.metrics import classification_report
import pandas as pd
import requests
import json
from tqdm import tqdm
import numpy as np
import time
tqdm.pandas()

In [2]:
def json_read(path: str):
    with open(path, encoding="utf-8") as fh:
        data = json.load(fh)
    return data

In [3]:
labels = pd.read_csv("../csv/20230807_103556.csv")
filenames = labels["File Name"].unique().tolist()
labels.head()

,File Name,File Id,Criterion Name,Expected Result,Actual Result,Report Time,Test Result NR,Batch Name,Expected Reason,Actual Reason
0,1684482083958_3770_810983451276_516462765.mp3,p-BLd4wpltpkEfsiZrlPIQ,Greeting,Yes,Yes,2023-08-07 09:57:54,Pass,Batch 300 call ngay 7/8 lan 2,NaN,NaN
1,1684482083958_3770_810983451276_516462765.mp3,p-BLd4wpltpkEfsiZrlPIQ,Focus on PTP,NaN,NaN,2023-08-07 09:57:54,Pass,Batch 300 call ngay 7/8 lan 2,KH cúp ngang,NaN
2,1684482083958_3770_810983451276_516462765.mp3,p-BLd4wpltpkEfsiZrlPIQ,Will you pay?,Yes,Yes,2023-08-07 09:57:54,Pass,Batch 300 call ngay 7/8 lan 2,NaN,NaN
3,1684482083958_3770_810983451276_516462765.mp3,p-BLd4wpltpkEfsiZrlPIQ,COC,Yes,Yes,2023-08-07 09:57:54,Pass,Batch 300 call ngay 7/8 lan 2,NaN,NaN
4,1684482083958_3770_810983451276_516462765.mp3,p-BLd4wpltpkEfsiZrlPIQ,Save result,Yes,No,2023-08-07 09:57:54,Fail,Batch 300 call ngay 7/8 lan 2,"KH từ chối, agent chưa nói oh1, kh cúp ngang =...","Lưu sai, không lưu result theo quy định - Khôn..."


In [5]:
call_data = pd.read_json("../json/data/m3_call_clean.json")
call_data = call_data[call_data["name"].isin(filenames)]
call_data.sample()

,name,agentChannel,content,metadata
399,1685925165925_3729_810963322393_519189914.mp3,1,"[{'channel': 1, 'text': 'chào linh gọi từ home...","{'parent': 'metadata_20230605_1686546722121', ..."


In [12]:
project_info = json_read("../json/project_info/m3_project_info_test.json")["project_info"]
criteria_name = ["willpaySummary", "nopaySummary", "paidSummary"]

In [7]:
def test_criteria(project_info, criteria_name, transcript, metadata, agent_channel, file_name="test"):
    url = "http://103.176.146.250:5005/predict/dialogue/test"
    payload = json.dumps({
      "project_info": project_info,
      "transcript": transcript,
      "metadata": metadata,
      "criteria": criteria_name,
      "fileName": file_name,
      "agentChannel": agent_channel
    })
    headers = {
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json()

def calculate_criteira(rec, wc_project_info, criteria_name):
    while True:
        try:
            result = test_criteria(wc_project_info, criteria_name,
                           rec["content"], rec["metadata"],
                           rec["agentChannel"], file_name=rec["name"])
            for criteria in criteria_name:
                rec[criteria] = result[criteria]["evaluate"]
            break
        except:
            print(result.keys())
            print(rec["name"])
            time.sleep(10)
    return rec

In [8]:
sample = call_data.sample().to_dict("records")[0]
result = test_criteria(project_info, criteria_name, sample["content"], sample["metadata"], sample["agentChannel"], file_name=sample["name"])

In [9]:
call_data.progress_apply(lambda rec: calculate_criteira(rec, project_info, criteria_name), axis=1)

 43%|██████████████████████████████████▍                                             | 129/300 [00:36<00:46,  3.64it/s]

dict_keys(['error'])
1685667093286_3600_850353933905_518718618.mp3


 47%|█████████████████████████████████████▊                                          | 142/300 [00:50<00:48,  3.25it/s]

dict_keys(['error'])
1684552745540_3737_810379211179_516546901.mp3


 48%|██████████████████████████████████████▏                                         | 143/300 [01:00<08:44,  3.34s/it]

...

dict_keys(['error'])
1685065025516_3730_810919274306_517547091.mp3


 93%|██████████████████████████████████████████████████████████████████████████▋     | 280/300 [02:10<01:35,  4.77s/it]

dict_keys(['error'])
1685583739973_3725_810767655515_518583039.mp3


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [02:26<00:00,  2.04it/s]


,name,agentChannel,content,metadata,willpaySummary,nopaySummary,paidSummary
0,1685157935457_3738_850961387032_517774092.mp3,1,"[{'channel': 1, 'text': 'cái gì xin chào chào ...","{'parent': 'metadata_20230527_1685945582075', ...",yes,no,no
1,1685152174765_3764_810989107939_517731851.mp3,1,"[{'channel': 1, 'text': 'xin chào bên em gọi q...","{'parent': 'metadata_20230527_1685945582075', ...",no,yes,no
2,1685158598464_3738_850972579342_522808569.mp3,1,"[{'channel': 1, 'text': 'dạ xin chào bên home ...","{'parent': 'metadata_20230527_1685945582075', ...",no,yes,no
3,1685581769516_3600_850988632007_518568283.mp3,1,"[{'channel': 1, 'text': 'alo chào em bên home ...","{'parent': 'metadata_20230601_1685945569381', ...",no,yes,no
4,1685851451096_3732_810912509169_519117581.mp3,1,"[{'channel': 1, 'text': 'chào phương home cred...","{'parent': 'metadata_20230604_1686546696463', ...",yes,yes,no
...,...,...,...,...,...,...,...
402,1685767446133_3466_810906924291_518985855.mp3,1,"[{'channel': 1, 'text': 'dạ alo chào nay mặt t...","{'parent': 'metadata_20230603_1686546710164', ...",no,no,no
407,1685927864453_3729_810974579084_519201989.mp3,1,"[{'channel': 1, 'text': 'chào linh gọi từ home...","{'parent': 'metadata_20230605_1686546722121', ...",no,yes,no
411,1684981476107_3771_850966858098_522434696.mp3,1,"[{'channel': 2, 'text': 'alo'}, {'channel': 1,...","{'parent': 'metadata_20230525_1685360595626', ...",no,yes,no
412,1685064372290_3743_810931516538_517542970.mp3,1,"[{'channel': 1, 'text': 'chào thi home credit ...","{'parent': 'metadata_20230526_1685360610102', ...",yes,yes,no


In [10]:
call_data

,name,agentChannel,content,metadata
0,1685157935457_3738_850961387032_517774092.mp3,1,"[{'channel': 1, 'text': 'cái gì xin chào chào ...","{'parent': 'metadata_20230527_1685945582075', ..."
1,1685152174765_3764_810989107939_517731851.mp3,1,"[{'channel': 1, 'text': 'xin chào bên em gọi q...","{'parent': 'metadata_20230527_1685945582075', ..."
2,1685158598464_3738_850972579342_522808569.mp3,1,"[{'channel': 1, 'text': 'dạ xin chào bên home ...","{'parent': 'metadata_20230527_1685945582075', ..."
3,1685581769516_3600_850988632007_518568283.mp3,1,"[{'channel': 1, 'text': 'alo chào em bên home ...","{'parent': 'metadata_20230601_1685945569381', ..."
4,1685851451096_3732_810912509169_519117581.mp3,1,"[{'channel': 1, 'text': 'chào phương home cred...","{'parent': 'metadata_20230604_1686546696463', ..."
...,...,...,...,...
402,1685767446133_3466_810906924291_518985855.mp3,1,"[{'channel': 1, 'text': 'dạ alo chào nay mặt t...","{'parent': 'metadata_20230603_1686546710164', ..."
407,1685927864453_3729_810974579084_519201989.mp3,1,"[{'channel': 1, 'text': 'chào linh gọi từ home...","{'parent': 'metadata_20230605_1686546722121', ..."
411,1684981476107_3771_850966858098_522434696.mp3,1,"[{'channel': 2, 'text': 'alo'}, {'channel': 1,...","{'parent': 'metadata_20230525_1685360595626', ..."
412,1685064372290_3743_810931516538_517542970.mp3,1,"[{'channel': 1, 'text': 'chào thi home credit ...","{'parent': 'metadata_20230526_1685360610102', ..."


In [15]:
compare = labels.loc[labels["Criterion Name"] == "Contain all information"].merge(call_data, left_on='File Name', right_on='name')
compare = compare[["name", "Criterion Name", "Expected Result", "Actual Result"] + criteria_name]
compare['Expected Result'] = compare['Expected Result'].replace(np.nan, "no")
compare["Expected Result"] = compare["Expected Result"].apply(str.lower)

KeyError: "['willpaySummary', 'nopaySummary', 'paidSummary'] not in index"

In [ ]:
y_true = [i for i in compare['Expected Result']]
y_pred_willpay = [i for i in compare['willpaySummary']]

In [ ]:
target_names = ['yes', 'no', 'partially']

In [ ]:
print("willpay:")
print(classification_report(y_true, y_pred_willpay, target_names=target_names, zero_division=1))

In [ ]:
y_pred_nopay = [i for i in compare['nopaySummary']]
print("nopay:")
print(classification_report(y_true, y_pred_nopay, target_names=target_names, zero_division=1))

In [ ]:
y_pred_paid = [i for i in compare['paidSummary']]
print("paid:")
print(classification_report(y_true, y_pred_paid, target_names=target_names, zero_division=1))